In [6]:
!pip install tensorflow==2.15.0

In [4]:
!pip install tensorflow-addons
!pip install tensorflow-addons==0.20.0
!pip install keras-cv

In [5]:
!pip install vit-keras
!pip install tensorflow keras vit-keras

In [7]:
!pip install transformers
!pip install kaggle

In [2]:
# !kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
# !unzip chest-xray-pneumonia.zip -d data

In [8]:
!mkdir -p data/train/Normal
!mkdir -p data/train/Pneumonia
!mkdir -p data/val/Normal
!mkdir -p data/val/Pneumonia
!mkdir -p data/train/Normal
!mkdir -p data/train/Pneumonia
!mkdir -p data/val/Normal
!mkdir -p data/val/Pneumonia
!mkdir -p data/test/Normal
!mkdir -p data/test/Pneumonia
!mkdir -p data/val/Pneumonia
!mkdir -p data/test/Normal
!mkdir -p data/test/Pneumonia


In [3]:
# import kagglehub
# # Download latest version
# path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")
# print("Path to dataset files:", path)

In [22]:
# Import Libraries
import os
import zipfile
import glob
import numpy as np
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, Concatenate, Reshape, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Add
import matplotlib.pyplot as plt

In [15]:
# Step 1: Install and configure Kaggle API
!pip install kaggle --upgrade --quiet

print("Please upload your kaggle.json file.")
uploaded = files.upload()

# Move kaggle.json to the appropriate directory
os.makedirs("/root/.kaggle/", exist_ok=True)
!cp kaggle.json /root/.kaggle/kaggle.json
os.chmod("/root/.kaggle/kaggle.json", 600)

# Step 2: Download dataset using Kaggle API
dataset_name = "paultimothymooney/chest-xray-pneumonia"  # Change for another dataset
download_path = "/content/"
print(f"Downloading dataset '{dataset_name}'...")
!kaggle datasets download -d {dataset_name} -p {download_path}

# Step 3: Extract dataset
zip_file = os.path.join(download_path, "chest-xray-pneumonia.zip")
extraction_path = os.path.join(download_path, "chest_xray")
print("Extracting the dataset...")
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)
print("Extraction complete.")

# Step 4: Organize dataset structure
source_dir = os.path.join(extraction_path, "chest_xray")
train_dir = os.path.join(extraction_path, "train")
val_dir = os.path.join(extraction_path, "val")
test_dir = os.path.join(extraction_path, "test")

for subdir in ['train', 'val', 'test']:
    for class_name in ['NORMAL', 'PNEUMONIA']:
        dest_path = os.path.join(
            train_dir if subdir == 'train' else val_dir if subdir == 'val' else test_dir, class_name
        )
        os.makedirs(dest_path, exist_ok=True)


Please upload your kaggle.json file.


Saving kaggle.json to kaggle (2).json
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)
Extracting the dataset...
Extraction complete.


In [17]:
# Move files into corresponding folders
for subdir in ['train', 'val', 'test']:
    for class_name in ['NORMAL', 'PNEUMONIA']:
        source_path = os.path.join(source_dir, subdir, class_name.upper())
        dest_path = os.path.join(
            train_dir if subdir == 'train' else val_dir if subdir == 'val' else test_dir, class_name
        )
        if os.path.exists(source_path):
            for file in os.listdir(source_path):
                if file.endswith('.jpeg'):  # Adjust for other image formats
                    # Check if the file already exists in the destination
                    destination_file = os.path.join(dest_path, file)
                    if not os.path.exists(destination_file):
                        # Only move the file if it doesn't exist in the destination
                        shutil.move(os.path.join(source_path, file), dest_path)

# Verify directory structure
for subdir in ['train', 'val', 'test']:
    for class_name in ['NORMAL', 'PNEUMONIA']:
        path = os.path.join(
            train_dir if subdir == 'train' else val_dir if subdir == 'val' else test_dir, class_name
        )
        print(f'{path}: {len(os.listdir(path))} files')

/content/chest_xray/train/NORMAL: 1341 files
/content/chest_xray/train/PNEUMONIA: 3875 files
/content/chest_xray/val/NORMAL: 8 files
/content/chest_xray/val/PNEUMONIA: 8 files
/content/chest_xray/test/NORMAL: 234 files
/content/chest_xray/test/PNEUMONIA: 390 files


In [19]:

# Step 5: Data preprocessing
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
val_data = datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical')
test_data = datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)

num_classes = train_data.num_classes


# Step 6: Define Vision Transformer block
def vision_transformer_block(inputs, num_heads=4, key_dim=64, ff_dim=128, dropout=0.1):
    x = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(inputs, inputs)
    x = Dropout(dropout)(x)
    x = Add()([x, inputs])
    x = LayerNormalization()(x)

    ff = Dense(ff_dim, activation="relu")(x)
    ff = Dropout(dropout)(ff)
    ff = Dense(inputs.shape[-1])(ff)

    x = Add()([ff, x])
    x = LayerNormalization()(x)
    return x


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [20]:

# Step 7: Build hybrid CNN + ViT model
def build_hybrid_model(input_shape=(224, 224, 3), num_classes=2):
    resnet_base = ResNet50(include_top=False, weights='imagenet', input_tensor=Input(shape=input_shape))
    resnet_base.trainable = False

    cnn_features = GlobalAveragePooling2D()(resnet_base.output)
    cnn_features = Dense(128, activation='relu')(cnn_features)
    cnn_features = Dropout(0.3)(cnn_features)

    vit_input = Input(shape=(16, 16, 64))
    reshaped_input = Reshape((16 * 16, 64))(vit_input)
    vit_features = vision_transformer_block(reshaped_input)
    vit_features = GlobalAveragePooling2D()(Reshape((16, 16, 64))(vit_features))

    combined_features = Concatenate()([cnn_features, vit_features])
    output = Dense(num_classes, activation='softmax')(combined_features)

    return Model(inputs=[resnet_base.input, vit_input], outputs=output)

hybrid_model = build_hybrid_model(input_shape=(224, 224, 3), num_classes=num_classes)
hybrid_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

# Step 8: Define data generator for ViT inputs
def get_vit_input(image_batch):
    return tf.random.normal([image_batch.shape[0], 16, 16, 64])  # Replace with real logic if needed

def hybrid_generator(image_gen):
    while True:
        image_batch, labels = next(image_gen)
        vit_input = get_vit_input(image_batch)
        yield ((image_batch, vit_input), labels)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [21]:

# Step 9: Train the model
train_gen = hybrid_generator(train_data)
val_gen = hybrid_generator(val_data)

history = hybrid_model.fit(
    train_gen,
    steps_per_epoch=len(train_data),
    validation_data=val_gen,
    validation_steps=len(val_data),
    epochs=5,  # Adjust epochs as needed
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
)

# Step 10: Evaluate the model
test_gen = hybrid_generator(test_data)
results = hybrid_model.evaluate(test_gen, steps=len(test_data))
print(f"Test Accuracy: {results[1] * 100:.2f}%")


Epoch 1/5
163/163 [==============================] - 1123s 7s/step - loss: 0.6483 - accuracy: 0.7030 - val_loss: 0.9179 - val_accuracy: 0.5000
Epoch 2/5
163/163 [==============================] - 1093s 7s/step - loss: 0.5735 - accuracy: 0.7427 - val_loss: 0.8558 - val_accuracy: 0.5000
Epoch 3/5
163/163 [==============================] - 1096s 7s/step - loss: 0.5409 - accuracy: 0.7435 - val_loss: 0.8385 - val_accuracy: 0.5000
Epoch 4/5
163/163 [==============================] - 1096s 7s/step - loss: 0.5232 - accuracy: 0.7437 - val_loss: 0.7703 - val_accuracy: 0.5000
Epoch 5/5
20/20 [==============================] - 123s 6s/step - loss: 0.5982 - accuracy: 0.6298
Test Accuracy: 62.98%
